# Multi-step traversal smoke test

Generate a small multi-step dataset, run the 1.5B reasoning model, and print prompts, responses, and grading stats.

In [13]:
import sys
from pathlib import Path

def _find_repo_root(start: Path) -> Path:
    markers = {"AGENTS.md", "environment.yml"}
    current = start.resolve()
    for parent in [current] + list(current.parents):
        if any((parent / marker).exists() for marker in markers):
            return parent
    return start.resolve()

PROJECT_ROOT = _find_repo_root(Path.cwd())
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))


from cutter.scripts.create_dataset import generate_examples
from cutter.scripts.generate_responses import run_batch
from cutter.utils.shared.basic import parse_range_arg
from cutter.utils.tree.encoding import DEVICE, load_reasoning_model, set_global_seed

DEPTH_RANGE = [2]
STEPS_RANGE = [2]
MIN_DEPTH, MAX_DEPTH = parse_range_arg(DEPTH_RANGE, "depth-range", min_value=1)
MIN_STEPS, MAX_STEPS = parse_range_arg(STEPS_RANGE, "steps-range", min_value=1)
DEPTHS = range(MIN_DEPTH, MAX_DEPTH + 1)
STEPS = list(range(MIN_STEPS, MAX_STEPS + 1))
NUM_SAMPLES = 5
SEED = 0
MODEL_ID = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
MAX_NEW_TOKENS = 2000

set_global_seed(SEED)


In [14]:
examples = generate_examples(
    depths=DEPTHS,
    num_samples=NUM_SAMPLES,
    seed=SEED,
    steps_range=STEPS,
)
dataset = examples
print(f"Loaded {len(dataset)} examples (in-memory)")


Loaded 5 examples (in-memory)


In [15]:
from pathlib import Path
tokenizer, model = load_reasoning_model(MODEL_ID, device=DEVICE, use_half_precision=True)
model.eval()

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 1536)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear(in_features=1536, out_features=1536, bias=True)
          (k_proj): Linear(in_features=1536, out_features=256, bias=True)
          (v_proj): Linear(in_features=1536, out_features=256, bias=True)
          (o_proj): Linear(in_features=1536, out_features=1536, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=1536, out_features=8960, bias=False)
          (up_proj): Linear(in_features=1536, out_features=8960, bias=False)
          (down_proj): Linear(in_features=8960, out_features=1536, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((1536,), eps=1e-06)
    (rotar

In [16]:
for idx, (result, _) in enumerate(
    run_batch(dataset, tokenizer, model, max_new_tokens=MAX_NEW_TOKENS, dataset_path=Path("in-memory"))
):
    print(f"Example {idx}")
    print("Prompt:\n" + result.record.prompt)
    print("Response:\n" + result.raw_response)
    target_path = " ".join(str(x) for x in result.record.path)
    parsed_path = " ".join(str(x) for x in result.parsed_path)
    print(f"Target path: {target_path}")
    print(f"Parsed path: {parsed_path}")
    print(
        "Exact: "
        + str(result.exact_match)
        + " | Partial: "
        + f"{result.partial_score:.3f}"
        + " | Prefix: "
        + f"{result.prefix_match}/{result.target_len}"
    )
    print("-" * 80)

Example 0
Prompt:
You are a path-finding assistant for a complete binary tree.
Tree depth: 2 (root depth 0; leaves depth 2).
Nodes are labeled uniquely as shown in the diagram (root at the top, then left to right by level).

Tree Graph:
            6
      2           4
   1     3     5     0

Task: Provide the shortest path from node 6 to 4 to 0, moving only along tree edges.
Guidelines:
- You may reason step by step, but end with a single line that contains only the path.
- Format the final line as `PATH: n0 n1 ... nk` using space-separated node ids.
- Use only node numbers from the tree diagram and do not add extra text after the final line.
Example format (depth 1): PATH: 1 0 2

Response:
Okay, so I need to find the shortest path from node 6 to node 0 in this binary tree. Let me visualize the tree first based on the diagram provided. The root is node 6, and it has two children: node 2 and node 4. Then, node 2 has two children: node 1 and node 3, and node 4 has two children: node 5 

/Users/cutterdawes/Desktop/Research/SPAR/HierarchicalProbes/cutter/scripts/generate_responses.py:201: UserWarning: Embedding extraction failed for example 4: Response tokenization mismatch with generated tokens
  warnings.warn(f"Embedding extraction failed for example {example_id}: {exc}")
